    Ben Christensen
    Math 402
    November 30, 2018


In [40]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split, KFold
from itertools import combinations

## 4.12

In [2]:
df = pd.read_csv("wages.csv")
df.head(n=10)

,wage,educ,exper,tenure,nonwhite,female,married,numdep,smsa,northcen,...,trcommpu,trade,services,profserv,profocc,clerocc,servocc,lwage,expersq,tenursq
0,3.10,11,2,0,0,1,0,2,1,0,...,0,0,0,0,0,0,0,1.131402,4,0
1,3.24,12,22,2,0,1,1,3,1,0,...,0,0,1,0,0,0,1,1.175573,484,4
2,3.00,11,2,0,0,0,0,2,0,0,...,0,1,0,0,0,0,0,1.098612,4,0
3,6.00,8,44,28,0,0,1,0,1,0,...,0,0,0,0,0,1,0,1.791759,1936,784
4,5.30,12,7,2,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1.667707,49,4
5,8.75,16,9,8,0,0,1,0,1,0,...,0,0,0,1,1,0,0,2.169054,81,64
6,11.25,18,15,7,0,0,0,0,1,0,...,0,1,0,0,1,0,0,2.420368,225,49
7,5.00,12,5,3,0,1,0,0,1,0,...,0,0,0,0,1,0,0,1.609438,25,9
8,3.60,12,26,4,0,1,0,2,1,0,...,0,1,0,0,1,0,0,1.280934,676,16
9,18.18,17,22,21,0,0,1,0,1,0,...,0,0,0,0,1,0,0,2.900322,484,441


## (i)

In [3]:
x = df[["educ", "exper", "tenure", "married", "female"]]
x = sm.add_constant(x, prepend=False)
y = df["wage"]

In [4]:
mod = sm.OLS(y, x)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.368
Model:                            OLS   Adj. R-squared:                  0.362
Method:                 Least Squares   F-statistic:                     60.61
Date:                Fri, 30 Nov 2018   Prob (F-statistic):           1.02e-49
Time:                        10:10:17   Log-Likelihood:                -1312.3
No. Observations:                 526   AIC:                             2637.
Df Residuals:                     520   BIC:                             2662.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
educ           0.5557      0.050     11.144      0.0

The coefficient on female decreases in absolute value. Therefore the other variables I added to the model are correlated with female and wage. Omitting them caused omitted variable bias.

R-squared increased significantly from .116 to .368. This is meaningful because this model explains more of the variability in the data. Education, tenure, and female are all significant to the .001 confidence level. Married is significant to a 90% confidence and exper is not significant to a 90% confidence standard.

## (ii)

In [5]:
df["married*female"] = df["married"]*df["female"]

In [6]:
x = df[["educ", "exper", "tenure", "married", "female", "married*female"]]
x = sm.add_constant(x, prepend=False)
y = df["wage"]
mod = sm.OLS(y, x)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.392
Model:                            OLS   Adj. R-squared:                  0.385
Method:                 Least Squares   F-statistic:                     55.84
Date:                Fri, 30 Nov 2018   Prob (F-statistic):           3.90e-53
Time:                        10:10:17   Log-Likelihood:                -1302.1
No. Observations:                 526   AIC:                             2618.
Df Residuals:                     519   BIC:                             2648.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
educ               0.5529      0.049     11.

The coefficient on female decreases even more in absolute value. The coefficient on marriedxfemale is -2.3953. There is possibly wage discrimination against women who are married. It's less likely to occur against single women. (Some research in labor economics shows the wage differences between men and women is due to preferences in flexibility of work schedule. Additional research shows occupation choice explains much of the difference). 

Education and tenure are still significant with 99.9% confidence. Now married and femalexmarried are also significant at that level. Female is no longer significant with even  60% confidence. Experience still does not satisfy significance with 90% confidence.

The interaction term requires a little nuance in interpreting the results. Ceterus paribus, the difference between a single man and a married woman would be + 1.8219 - 0.3085 - 2.3953 = -0.8819 .

## 4.13

In [8]:
x = df[['female','educ','exper','tenure','married','married*female','numdep','nonwhite']]
x = sm.add_constant(x)
y = df["wage"]
mod = sm.OLS(y, x)
res=mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.393
Model:                            OLS   Adj. R-squared:                  0.384
Method:                 Least Squares   F-statistic:                     41.90
Date:                Fri, 30 Nov 2018   Prob (F-statistic):           1.42e-51
Time:                        10:10:36   Log-Likelihood:                -1301.6
No. Observations:                 526   AIC:                             2621.
Df Residuals:                     517   BIC:                             2660.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -2.5664      0.780     -3.

In [9]:
x = df[['female','educ','exper','tenure','married','married*female','numdep']]
x = sm.add_constant(x)
y = df["wage"]
mod = sm.OLS(y, x)
res=mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.393
Model:                            OLS   Adj. R-squared:                  0.385
Method:                 Least Squares   F-statistic:                     47.92
Date:                Fri, 30 Nov 2018   Prob (F-statistic):           2.23e-52
Time:                        10:10:58   Log-Likelihood:                -1301.7
No. Observations:                 526   AIC:                             2619.
Df Residuals:                     518   BIC:                             2654.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -2.6032      0.776     -3.

In [10]:
x = df[['female','educ','exper','tenure','married','married*female']]
x = sm.add_constant(x)
y = df["wage"]
mod = sm.OLS(y, x)
res=mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.392
Model:                            OLS   Adj. R-squared:                  0.385
Method:                 Least Squares   F-statistic:                     55.84
Date:                Fri, 30 Nov 2018   Prob (F-statistic):           3.90e-53
Time:                        10:11:22   Log-Likelihood:                -1302.1
No. Observations:                 526   AIC:                             2618.
Df Residuals:                     519   BIC:                             2648.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -2.3942      0.730     -3.

In [12]:
x = df[['educ','exper','tenure','married','married*female']]
x = sm.add_constant(x)
y = df["wage"]
mod = sm.OLS(y, x)
res=mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.392
Model:                            OLS   Adj. R-squared:                  0.386
Method:                 Least Squares   F-statistic:                     66.95
Date:                Fri, 30 Nov 2018   Prob (F-statistic):           5.98e-54
Time:                        10:11:44   Log-Likelihood:                -1302.4
No. Observations:                 526   AIC:                             2617.
Df Residuals:                     520   BIC:                             2642.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -2.5838      0.685     -3.

In [13]:
x = df[['educ','tenure','married','married*female']]
x = sm.add_constant(x)
y = df["wage"]
mod = sm.OLS(y, x)
res=mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.389
Model:                            OLS   Adj. R-squared:                  0.384
Method:                 Least Squares   F-statistic:                     82.83
Date:                Fri, 30 Nov 2018   Prob (F-statistic):           2.10e-54
Time:                        10:11:55   Log-Likelihood:                -1303.6
No. Observations:                 526   AIC:                             2617.
Df Residuals:                     521   BIC:                             2639.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -2.0888      0.610     -3.

I just iteratively removed the feature with the highest p-value until all the features satisfy 95% confidence. Except for the final iteration, adjusted R-squared increased every time.

## 4.14

In [22]:
features = ['female','educ','exper','tenure','married','married*female','numdep','nonwhite']
#Prime AIC and BIC with using all features
smallest_AIC = 2621.
smallest_BIC = 2660.
best_aic_subset, best_bic_subset = features, features
for num_features in range(1, len(features)+1):
    for subset in combinations(features, num_features):
        if num_features == 1:
            subset = [subset[0]]
        else:
            subset = list(subset)
        x = df[subset]
        x = sm.add_constant(x)
        y = df["wage"]
        model = sm.OLS(y, x).fit()
        if model.aic < smallest_AIC:
            smallest_AIC = model.aic
            best_aic_subset = subset
        if model.bic < smallest_BIC:
            smallest_BIC = model.bic
            best_bic_subset = subset
print("Features that give optimal AIC:")
print(best_aic_subset)
print("AIC:", smallest_AIC)
print("\nFeatures that give optimal BIC:")
print(best_bic_subset)
print("BIC:", smallest_BIC)

Features that give optimal AIC:
['educ', 'exper', 'tenure', 'married', 'married*female']
AIC: 2616.70234852

Features that give optimal BIC:
['educ', 'tenure', 'married', 'married*female']
BIC: 2638.53368103


## 4.15

In [29]:
regr = linear_model.LinearRegression()
X = df[['female','educ','exper','tenure','married','married*female','numdep','nonwhite']]
y = df['wage']
regr.fit(X,y)
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
model.params
np.allclose(model.params[1:], regr.coef_)

True

The coefficients are the same across the two methods.

## 4.16

In [31]:
regr1, regr2 = linear_model.LinearRegression(), linear_model.LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X2 = df[['educ','tenure','married','married*female']]
X2_train, X2_test = train_test_split(X2, y, test_size=0.30, random_state=42)[:2]
regr1.fit(X_train, y_train)
regr2.fit(X2_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [37]:
print("MSE for model 1:", regr1.score(X_test, y_test))
print("MSE for model 2:", regr2.score(X2_test, y_test))

MSE for model 1: 0.367215035919
MSE for model 2: 0.373546929158


Model 1 has better MSE.

## 4.17

In [47]:
kf = KFold(n_splits=7, shuffle=True, random_state=16)
MSEs_1, MSEs_2 = list(), list()
for train_index, test_index in kf.split(X):
    regr1, regr2 = linear_model.LinearRegression(), linear_model.LinearRegression()
    X_train, X_test = X.values[train_index], X.values[test_index]
    X2_train, X2_test = X2.values[train_index], X2.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    regr1.fit(X_train, y_train)
    regr2.fit(X2_train, y_train)
    MSEs_1.append(regr1.score(X_test, y_test))
    MSEs_2.append(regr2.score(X2_test, y_test))
print("Model 1 average MSE:", np.mean(MSEs_1))
print("Model 2 average MSE:", np.mean(MSEs_2))

Model 1 average MSE: 0.347358242791
Model 2 average MSE: 0.351723210891


Model 1 has better MSE.